In [1]:
import sage.all
from collections.abc import Callable, Container
from copy import copy
from warnings import warn

from sage.structure.richcmp import richcmp_method, richcmp
from sage.combinat.combination import Combinations
from sage.combinat.posets.posets import FinitePoset
from sage.geometry.cone import (_ambient_space_point,
                                Cone,
                                ConvexRationalPolyhedralCone,
                                IntegralRayCollection,
                                is_Cone,
                                normalize_rays)
from sage.geometry.hasse_diagram import lattice_from_incidences
from sage.geometry.point_collection import PointCollection
from sage.geometry.toric_lattice import ToricLattice, is_ToricLattice
from sage.geometry.toric_plotter import ToricPlotter
from sage.graphs.digraph import DiGraph
from sage.matrix.all import matrix
from sage.misc.cachefunc import cached_method
from sage.misc.all import walltime, prod
from sage.modules.all import vector, span
from sage.rings.all import QQ, ZZ
from sage.functions.other import floor, ceil

import time

In [2]:
import time
def timeit(func):
    def wrapper(*args, **kw):
        tik = time.time()
        func(*args, **kw)
        tok = time.time()
        print("time is {}".format(tok-tik))
    return wrapper


In [3]:
from sage.geometry.fan import RationalPolyhedralFan

In [4]:
@richcmp_method
class RationalPolyhedralFan_rtree(RationalPolyhedralFan):
    r"""
    Create a pointed rational polyhedral fan with rtree data structure, 
    which reduces the time for determining whether a point is in the support 
    of the fan or not.

    .. WARNING::

        This class does not perform any checks of correctness of input nor
        does it convert input into the standard representation. Use
        :func:`Fan` to construct fans from "raw data" or :func:`FaceFan` and
        :func:`NormalFan` to get fans associated to polytopes.

    Fans are immutable, but they cache most of the returned values.

    INPUT:

    - ``cones`` -- list of generating cones of the fan, each cone given as a
      list of indices of its generating rays in ``rays``;

    - ``rays`` -- list of immutable primitive vectors in ``lattice``
      consisting of exactly the rays of the fan (i.e. no "extra" ones);

    - ``lattice`` -- :class:`ToricLattice
      <sage.geometry.toric_lattice.ToricLatticeFactory>`, `\ZZ^n`, or any
      other object that behaves like these. If ``None``, it will be determined
      as :func:`parent` of the first ray. Of course, this cannot be done if
      there are no rays, so in this case you must give an appropriate
      ``lattice`` directly;

    - ``is_complete`` -- if given, must be ``True`` or ``False`` depending on
      whether this fan is complete or not. By default, it will be determined
      automatically if necessary;

    - ``virtual_rays`` -- if given, must be a list of immutable primitive
      vectors in ``lattice``, see :meth:`virtual_rays` for details. By default,
      it will be determined automatically if necessary.

    OUTPUT:

    - a rational polyhedral fan with rtree if it is pointed, or a rational 
        polyhedral fan without rtree if it is NOT pointed.
    """

    def __init__(self, cones, rays, lattice,
                 is_complete=None, virtual_rays=None):
        r"""
        See :class:`RationalPolyhedralFan_rtree` for documentation.

        TESTS::

            sage: v = vector([0,1])
            sage: v.set_immutable()
            sage: f = sage.geometry.fan.RationalPolyhedralFan(
            ....:                       [(0,)], [v], None)
            sage: f.rays()
            (0, 1)
            in Ambient free module of rank 2
            over the principal ideal domain Integer Ring
            sage: TestSuite(f).run()
            sage: f = Fan([(0,)], [(0,1)])
            sage: TestSuite(f).run()
        """
        #copy generator
        tik = time.time()
        fan_indices = list(cones)
        cones = (c_idx for c_idx in fan_indices)
        super().__init__(cones, rays, lattice, is_complete=None, virtual_rays=None)
        self._is_pointed = None
        
        # Determining whether the fan is pointed automatically. 
        
        if len(self._generating_cones) != 0:
            #Trying LP solver
            p = MixedIntegerLinearProgram(solver = 'GLPK', maximization = True)
            x = p.new_variable()
            b = p.new_variable()
            B = p.new_variable()
            p.solver_parameter("simplex_or_intopt", "simplex_only") # use simplex only
            p.set_objective(B[0])
            for i in range(len(rays)):
                p.add_constraint(p.sum(x[_]*rays[i][_] for _ in range(self.dim()))<= -b[i])
            p.add_constraint(1>= b[i] >= B[0])
            for _ in range(self.dim()):
                p.add_constraint( -1 <= x[_] <= 1 )
            p.add_constraint(1>=B[0]>=0)

            try:
                # Verifying whether it is a normal vector or not
                if p.solve() > 10e-15:
                    temp_x = vector(p.get_values(x))
                    if all(temp_x.dot_product(ray)< 0 for ray in rays):
                        # Been verified, thus, the fan is pointed.
                        self._normal_vector = temp_x     
                        self._is_pointed = True
            except sage.numerical.mip.MIPSolverException:
                pass
            # If the LP-solver fails, we may use the following facets-based method.
            if not self._is_pointed:
                # Trying Facets-based method.
                # Facets-based method, computing the common intersection of all polar 
                # cones of the generating cones in the fan. if the common intersection 
                # is empty, then it is not pointed; otherwise, it is pointed.
                empty_cone = Cone([],lattice = lattice)
                common_intersection = empty_cone.dual()
                for c in self._generating_cones:
                    common_intersection = common_intersection.intersection(c.polar())
                    if common_intersection.dim() == 0:
                        # If common_intersection is empty, then the fan is not pointed.
                        self._is_pointed = False
                        break
                if self._is_pointed == None:
                    temp_x = sum(vector(ray, QQ)/vector(ray, QQ).norm(infinity) for ray in common_intersection.rays())  
                    if all(temp_x.dot_product(ray)< 0 for ray in rays):
                        # Been verified, thus, the fan is pointed.
                        self._normal_vector = temp_x      
                        self._is_pointed = True 
        else:
            # An empty fan is considered as pointed, but without a normal vector.
            self._is_pointed = True

        # If the fan is pointed, then we build the rtree for it. ``self._has_rtree`` is a 
        # trigger for the usage of rtree. Since rtree package can only store 2 to high 
        # dimensional boxes, we give restriction below.
        self._has_rtree = False
        if self._is_pointed and self.dim()>=2 and len(self._generating_cones)>0:
            # Generating a random family of directions for projections, by 
            # neuralization of inner product of a direction with the normal vector.
            self._has_rtree = True
            self._normal_vector = vector(self._normal_vector)
            self._directions = []
            ndirections = self.dim()
            # Finding a coordinate that is non-zero of the normal vector.
            for i in range(self.dim()):
                if self._normal_vector[i] != 0:
                    coord = i
                    break

            for i in range(ndirections):
                direction = vector([choice([-1,1])*randint(1,127) for i in range(self.dim())], QQ)
                diff = direction.dot_product(self._normal_vector)
                direction[coord] = direction[coord] - diff/self._normal_vector[coord]
                self._directions.append(direction)
            table = []
            for (i,d) in enumerate(self._directions):
                t_i = []
                abs_max = 0
                for r in rays:
                    val = r.dot_product(d)/r.dot_product(self._normal_vector)
                    if abs_max < abs(val):
                        abs_max = abs(val)
                    t_i.append(val)
                if abs_max > 10e-15:
                    ratio = 2**26/ceil(abs_max)
                    self._directions[i] = ratio * self._directions[i]
                    t_i = [val * ratio for val in t_i]
                table.append(t_i)

            # The intersection between each cone and the separating hyperplane, which is -1 distance away
            # from the origin, is defined as the polytopical token for each cone.
            # Let r be the point that is the intersection between a ray and the separating hyperplane, x
            # be the unit vector of the normal vector, u be the unit vector of the ray, r = k*u, k>0.
            # We have <-x,u>= cos t = 1 / k. Thus, k = -1/<x,u>, r = k*u = -u/<x,u>. For any vector v 
            # whose direction is along with the ray r, v = j * u, because u = v/j, we have 
            # r = -(v/j)/<x,v/j>, = -v/<x,v>.
            tok = time.time()
            print("Before rtree{}s.".format(tok-tik))
            from rtree import index
            p = index.Property()
            p.dimension = ndirections
            self._rtree = index.Index(properties = p, interleaved = False)
            time_sum = 0
            for (i, indices) in enumerate(fan_indices):
                box = []
                for _ in range(ndirections):
                    proj_min, proj_max = infinity, -infinity
                    for idx in indices:
                        if table[_][idx] < proj_min:
                            proj_min = table[_][idx]
                        if table[_][idx] > proj_max:
                            proj_max = table[_][idx]
                    box = box + self._adjusting_gauge(proj_min, proj_max)
                self._rtree.insert(0, box, obj = i)

            
    def _adjusting_gauge(self, lower, upper):
        # This function is used for inserting boxes in rtree. Since rtree has limited precision, the rational 
        # interval has to been converted to a compatible integer interval.
        # The domain of rtree only consists of -\infty, integers in [-2^52,2^52], +\infty.
        prec_range = (-2**52, 2**52)
        if lower<=prec_range[0]:
            x_k = -infinity
        elif lower<prec_range[1]+1:
            x_k = floor(lower)
        else:
            x_k = infinity
        if upper<prec_range[0]-1:
            y_k = -infinity
        elif upper<=prec_range[1]:
            y_k = ceil(upper)
        else:
            y_k = infinity
        return [x_k, y_k]


    def is_pointed(self):
        return self._is_pointed
    
    def support_contains(self, *args):
        r"""
        Check if a point is contained in the support of the fan.

        The support of a fan is the union of all cones of the fan. If
        you want to know whether the fan contains a given cone, you
        should use :meth:`contains` instead.

        INPUT:

        - ``*args`` -- an element of ``self.lattice()`` or something
          that can be converted to it (for example, a list of
          coordinates).

        OUTPUT:

        - ``True`` if ``point`` is contained in the support of the
          fan, ``False`` otherwise.

        TESTS::

            sage: cone1 = Cone([(0,-1), (1,0)])
            sage: cone2 = Cone([(1,0), (0,1)])
            sage: f = Fan([cone1, cone2])

        We check if some points are in this fan::

            sage: f.support_contains(f.lattice()(1,0))
            True
            sage: f.support_contains(cone1)    # a cone is not a point of the lattice
            False
            sage: f.support_contains((1,0))
            True
            sage: f.support_contains(1,1)
            True
            sage: f.support_contains((-1,0))
            False
            sage: f.support_contains(f.lattice().dual()(1,0)) #random output (warning)
            False
            sage: f.support_contains(f.lattice().dual()(1,0))
            False
            sage: f.support_contains(1)
            False
            sage: f.support_contains(0)   # 0 converts to the origin in the lattice
            True
            sage: f.support_contains(1/2, sqrt(3))
            True
            sage: f.support_contains(-1/2, sqrt(3))
            False
        """

        if len(args) == 1:
            point = args[0]
        else:
            point = args
        try:
            point = _ambient_space_point(self, point)
        except TypeError as ex:
            if str(ex).endswith("have incompatible lattices!"):
                warn("you have checked if a fan contains a point "
                     "from an incompatible lattice, this is False!",
                     stacklevel=3)
            return False
        if self.is_complete():
            return True
        
        # Using rtree for overestimating.
        if self._has_rtree:
            r = vector(point)
            # See whether the point has a positive direction with the normal vector.
            if r.dot_product(self._normal_vector)>=0: 
                return False
            box = []
            for d in self._directions:
                val = r.dot_product(d)/r.dot_product(self._normal_vector)
                box = box + self._adjusting_gauge(val, val)
            if self._rtree.count(box) == 0:
                return False

        return any(point in cone for cone in self)


In [5]:
def Fan_rtree(cones, rays=None, lattice=None, check=True, normalize=True,
        is_complete=None, virtual_rays=None, discard_faces=False,
        allow_arrangement=False):
    r"""
    Construct a rational polyhedral fan.

    .. NOTE::

        Approximate time to construct a fan consisting of `n` cones is `n^2/5`
        seconds. That is half an hour for 100 cones. This time can be
        significantly reduced in the future, but it is still likely to be
        `\sim n^2` (with, say, `/500` instead of `/5`). If you know that your
        input does form a valid fan, use ``check=False`` option to skip
        consistency checks.

    INPUT:

    - ``cones`` -- list of either
      :class:`Cone<sage.geometry.cone.ConvexRationalPolyhedralCone>` objects
      or lists of integers interpreted as indices of generating rays in
      ``rays``. These must be only **maximal** cones of the fan, unless
      ``discard_faces=True`` or ``allow_arrangement=True`` option is specified;

    - ``rays`` -- list of rays given as list or vectors convertible to the
      rational extension of ``lattice``. If ``cones`` are given by
      :class:`Cone<sage.geometry.cone.ConvexRationalPolyhedralCone>` objects
      ``rays`` may be determined automatically. You still may give them
      explicitly to ensure a particular order of rays in the fan. In this case
      you must list all rays that appear in ``cones``. You can give "extra"
      ones if it is convenient (e.g. if you have a big list of rays for
      several fans), but all "extra" rays will be discarded;

    - ``lattice`` -- :class:`ToricLattice
      <sage.geometry.toric_lattice.ToricLatticeFactory>`, `\ZZ^n`, or any
      other object that behaves like these. If not specified, an attempt will
      be made to determine an appropriate toric lattice automatically;

    - ``check`` -- by default the input data will be checked for correctness
      (e.g. that intersection of any two given cones is a face of each),
      unless ``allow_arrangement=True`` option is specified. If you
      know for sure that the input is correct, you may significantly decrease
      construction time using ``check=False`` option;

    - ``normalize`` -- you can further speed up construction using
      ``normalize=False`` option. In this case ``cones`` must be a list of
      **sorted** :class:`tuples` and ``rays`` must be immutable primitive
      vectors in ``lattice``. In general, you should not use this option, it
      is designed for code optimization and does not give as drastic
      improvement in speed as the previous one;

    - ``is_complete`` -- every fan can determine on its own if it is complete
      or not, however it can take quite a bit of time for "big" fans with many
      generating cones. On the other hand, in some situations it is known in
      advance that a certain fan is complete. In this case you can pass
      ``is_complete=True`` option to speed up some computations. You may also
      pass ``is_complete=False`` option, although it is less likely to be
      beneficial. Of course, passing a wrong value can compromise the
      integrity of data structures of the fan and lead to wrong results, so
      you should be very careful if you decide to use this option;

    - ``virtual_rays`` -- (optional, computed automatically if needed) a list of
      ray generators to be used for :meth:`virtual_rays`;

    - ``discard_faces`` -- by default, the fan constructor expects the list of
      **maximal** cones, unless ``allow_arrangement=True`` option is specified.
      If you provide "extra" ones and leave ``allow_arrangement=False`` (default)
      and ``check=True`` (default), an exception will be raised.
      If you provide "extra" cones and set ``allow_arrangement=False`` (default)
      and ``check=False``, you may get wrong results as assumptions on internal
      data structures will be invalid. If you want the fan constructor to
      select the maximal cones from the given input, you may provide
      ``discard_faces=True`` option (it works both for ``check=True`` and
      ``check=False``).

    - ``allow_arrangement`` -- by default (``allow_arrangement=False``),
      the fan constructor expects that the intersection of any two given cones is
      a face of each. If ``allow_arrangement=True`` option is specified, then
      construct a rational polyhedralfan from the cone arrangement, so that the
      union of the cones in the polyhedral fan equals to the union of the given
      cones, and each given cone is the union of some cones in the polyhedral fan.

    OUTPUT:

    - a :class:`fan <RationalPolyhedralFan>`.

    .. SEEALSO::

        In 2 dimensions you can cyclically order the rays. Hence the
        rays determine a unique maximal fan without having to specify
        the cones, and you can use :func:`Fan2d` to construct this
        fan from just the rays.

    EXAMPLES:

    Let's construct a fan corresponding to the projective plane in several
    ways::

        sage: cone1 = Cone([(1,0), (0,1)])
        sage: cone2 = Cone([(0,1), (-1,-1)])
        sage: cone3 = Cone([(-1,-1), (1,0)])
        sage: P2 = Fan([cone1, cone2, cone2])
        Traceback (most recent call last):
        ...
        ValueError: you have provided 3 cones, but only 2 of them are maximal!
        Use discard_faces=True if you indeed need to construct a fan from
        these cones.

    Oops! There was a typo and ``cone2`` was listed twice as a generating cone
    of the fan. If it was intentional (e.g. the list of cones was generated
    automatically and it is possible that it contains repetitions or faces of
    other cones), use ``discard_faces=True`` option::

        sage: P2 = Fan([cone1, cone2, cone2], discard_faces=True)
        sage: P2.ngenerating_cones()
        2

    However, in this case it was definitely a typo, since the fan of
    `\mathbb{P}^2` has 3 maximal cones::

        sage: P2 = Fan([cone1, cone2, cone3])
        sage: P2.ngenerating_cones()
        3

    Looks better. An alternative way is ::

        sage: rays = [(1,0), (0,1), (-1,-1)]
        sage: cones = [(0,1), (1,2), (2,0)]
        sage: P2a = Fan(cones, rays)
        sage: P2a.ngenerating_cones()
        3
        sage: P2 == P2a
        False

    That may seem wrong, but it is not::

        sage: P2.is_equivalent(P2a)
        True

    See :meth:`~RationalPolyhedralFan.is_equivalent` for details.

    Yet another way to construct this fan is ::

        sage: P2b = Fan(cones, rays, check=False)
        sage: P2b.ngenerating_cones()
        3
        sage: P2a == P2b
        True

    If you try the above examples, you are likely to notice the difference in
    speed, so when you are sure that everything is correct, it is a good idea
    to use ``check=False`` option. On the other hand, it is usually **NOT** a
    good idea to use ``normalize=False`` option::

        sage: P2c = Fan(cones, rays, check=False, normalize=False)
        Traceback (most recent call last):
        ...
        AttributeError: 'tuple' object has no attribute 'parent'

    Yet another way is to use functions :func:`FaceFan` and :func:`NormalFan`
    to construct fans from :class:`lattice polytopes
    <sage.geometry.lattice_polytope.LatticePolytopeClass>`.

    We have not yet used ``lattice`` argument, since if was determined
    automatically::

        sage: P2.lattice()
        2-d lattice N
        sage: P2b.lattice()
        2-d lattice N

    However, it is necessary to specify it explicitly if you want to construct
    a fan without rays or cones::

        sage: Fan([], [])
        Traceback (most recent call last):
        ...
        ValueError: you must specify the lattice
        when you construct a fan without rays and cones!
        sage: F = Fan([], [], lattice=ToricLattice(2, "L"))
        sage: F
        Rational polyhedral fan in 2-d lattice L
        sage: F.lattice_dim()
        2
        sage: F.dim()
        0

    In the following examples, we test the ``allow_arrangement=True`` option.
    See :trac:`25122`.

    The intersection of the two cones is not a face of each. Therefore,
    they do not belong to the same rational polyhedral fan::

        sage: c1 = Cone([(-2,-1,1), (-2,1,1), (2,1,1), (2,-1,1)])
        sage: c2 = Cone([(-1,-2,1), (-1,2,1), (1,2,1), (1,-2,1)])
        sage: c1.intersection(c2).is_face_of(c1)
        False
        sage: c1.intersection(c2).is_face_of(c2)
        False
        sage: Fan([c1, c2])
        Traceback (most recent call last):
        ...
        ValueError: these cones cannot belong to the same fan!
        ...

    Let's construct the fan using ``allow_arrangement=True`` option::

        sage: fan = Fan([c1, c2], allow_arrangement=True)
        sage: fan.ngenerating_cones()
        5

    Another example where cone c2 is inside cone c1::

        sage: c1 = Cone([(4, 0, 0), (0, 4, 0), (0, 0, 4)])
        sage: c2 = Cone([(2, 1, 1), (1, 2, 1), (1, 1, 2)])
        sage: fan = Fan([c1, c2], allow_arrangement=True)
        sage: fan.ngenerating_cones()
        7
        sage: fan.plot()
        Graphics3d Object

    Cones of different dimension::

        sage: c1 = Cone([(1,0),(0,1)])
        sage: c2 = Cone([(2,1)])
        sage: c3 = Cone([(-1,-2)])
        sage: fan = Fan([c1, c2, c3], allow_arrangement=True)
        sage: for cone in sorted(fan.generating_cones()): print(sorted(cone.rays()))
        [N(-1, -2)]
        [N(0, 1), N(1, 2)]
        [N(1, 0), N(2, 1)]
        [N(1, 2), N(2, 1)]

    A 3-d cone and a 1-d cone::

        sage: c3 = Cone([[0, 1, 1], [1, 0, 1], [0, -1, 1], [-1, 0, 1]])
        sage: c1 = Cone([[0, 0, 1]])
        sage: fan1 = Fan([c1, c3], allow_arrangement=True)
        sage: fan1.plot()
        Graphics3d Object

    A 3-d cone and two 2-d cones::

        sage: c2v = Cone([[0, 1, 1], [0, -1, 1]])
        sage: c2h = Cone([[1, 0, 1], [-1, 0, 1]])
        sage: fan2 = Fan([c2v, c2h, c3], allow_arrangement=True)
        sage: fan2.is_simplicial()
        True
        sage: fan2.is_equivalent(fan1)
        True
    """
    def result():
        # "global" does not work here...
        R, V = rays, virtual_rays
        if V is not None:
            if normalize:
                V = normalize_rays(V, lattice)
            if check:
                R = PointCollection(V, lattice)
                V = PointCollection(V, lattice)
                d = lattice.dimension()
                if len(V) != d - R.dim() or (R + V).dim() != d:
                    raise ValueError("virtual rays must be linearly "
                    "independent and with other rays span the ambient space.")
        return RationalPolyhedralFan_rtree(cones, R, lattice, is_complete, V)

    if not check and not normalize and not discard_faces and not allow_arrangement:
        return result()
    if not isinstance(cones, list):
        try:
            cones = list(cones)
        except TypeError:
            raise TypeError(
                "cones must be given as an iterable!"
                "\nGot: %s" % cones)
    if not cones:
        if lattice is None:
            if rays is not None and rays:
                lattice = normalize_rays(rays, lattice)[0].parent()
            else:
                raise ValueError("you must specify the lattice when you "
                                 "construct a fan without rays and cones!")
        cones = ((), )
        rays = ()
        return result()
    if is_Cone(cones[0]):
        # Construct the fan from Cone objects
        if lattice is None:
            lattice = cones[0].lattice()
            # If we determine the lattice automatically, we do not
            # want to force any conversion. TODO: take into account
            # coercions?
            if check:
                for cone in cones:
                    if cone.lattice() != lattice:
                        raise ValueError("cones belong to different lattices "
                            "(%s and %s), cannot determine the lattice of the "
                            "fan!" % (lattice, cone.lattice()))
        for i, cone in enumerate(cones):
            if cone.lattice() != lattice:
                cones[i] = Cone(cone.rays(), lattice, check=False)
        if check:
            for cone in cones:
                if not cone.is_strictly_convex():
                    raise ValueError(
                                    "cones of a fan must be strictly convex!")
        # Optimization for fans generated by a single cone
        if len(cones) == 1 and rays is None:
            cone = cones[0]
            cones = (tuple(range(cone.nrays())), )
            rays = cone.rays()
            is_complete = lattice.dimension() == 0
            return result()
        if allow_arrangement:
            cones = _refine_arrangement_to_fan(cones)
            cones = _discard_faces(cones)
        elif check:
            # Maybe we should compute all faces of all cones and save them for
            # later if we are doing this check?
            generating_cones = []
            for cone in sorted(cones, key=lambda cone: cone.dim(),
                               reverse=True):
                is_generating = True
                for g_cone in generating_cones:
                    i_cone = cone.intersection(g_cone)
                    if i_cone.is_face_of(cone) and i_cone.is_face_of(g_cone):
                        if i_cone.dim() == cone.dim():
                            is_generating = False  # cone is a face of g_cone
                            break
                    else:
                        raise ValueError(
                                "these cones cannot belong to the same fan!"
                                "\nCone 1 rays: %s\nCone 2 rays: %s"
                                % (g_cone.rays(), cone.rays()))
                if is_generating:
                    generating_cones.append(cone)
            if len(cones) > len(generating_cones):
                if discard_faces:
                    cones = generating_cones
                else:
                    raise ValueError("you have provided %d cones, but only %d "
                        "of them are maximal! Use discard_faces=True if you "
                        "indeed need to construct a fan from these cones." %
                        (len(cones), len(generating_cones)))
        elif discard_faces:
            cones = _discard_faces(cones)
        ray_set = set([])
        for cone in cones:
            ray_set.update(cone.rays())
        if rays:    # Preserve the initial order of rays, if they were given
            rays = normalize_rays(rays, lattice)
            new_rays = []
            for ray in rays:
                if ray in ray_set and ray not in new_rays:
                    new_rays.append(ray)
            if len(new_rays) != len(ray_set):
                raise ValueError(
                  "if rays are given, they must include all rays of the fan!")
            rays = new_rays
        else:
            rays = tuple(sorted(ray_set))
        cones = (tuple(sorted(rays.index(ray) for ray in cone.rays()))
                 for cone in cones)
        return result()
    # Construct the fan from rays and "tuple cones"
    rays = normalize_rays(rays, lattice)
    for n, cone in enumerate(cones):
        try:
            cones[n] = sorted(cone)
        except TypeError:
            raise TypeError("cannot interpret %s as a cone!" % cone)
    if not check and not discard_faces and not allow_arrangement:
        return result()
    # If we do need to make all the check, build explicit cone objects first
    return Fan_rtree((Cone([rays[n] for n in cone], lattice) for cone in cones),
               rays, lattice, is_complete=is_complete,
               virtual_rays=virtual_rays, discard_faces=discard_faces,
               allow_arrangement=allow_arrangement)

def discard_faces(cones):
    r"""
    Return the cones of the given list which are not faces of each other.

    INPUT:

    - ``cones`` -- a list of
      :class:`cones <sage.geometry.cone.ConvexRationalPolyhedralCone>`.

    OUTPUT:

    - a list of
      :class:`cones <sage.geometry.cone.ConvexRationalPolyhedralCone>`,
      sorted by dimension in decreasing order.

    EXAMPLES:

    Consider all cones of a fan::

        sage: Sigma = toric_varieties.P2().fan()
        sage: cones = flatten(Sigma.cones())
        sage: len(cones)
        7

    Most of them are not necessary to generate this fan::

        sage: from sage.geometry.fan import discard_faces
        sage: len(discard_faces(cones))
        3
        sage: Sigma.ngenerating_cones()
        3
    """
    # Convert to a list or make a copy, so that the input is unchanged.
    cones = list(cones)
    cones.sort(key=lambda cone: cone.dim(), reverse=True)
    generators = []
    for cone in cones:
        if not any(cone.is_face_of(other) for other in generators):
            generators.append(cone)
    return generators


_discard_faces = discard_faces  # Due to a name conflict in Fan constructor


def _refine_arrangement_to_fan(cones):
    """
    Refine the cones of the given list so that they can belong to the same fan.

    INPUT:

    - ``cones`` -- a list of rational cones that are possibly overlapping.

    OUTPUT:

    - a list of refined cones.

    EXAMPLES::

        sage: from sage.geometry.fan import _refine_arrangement_to_fan
        sage: c1 = Cone([(-2,-1,1), (-2,1,1), (2,1,1), (2,-1,1)])
        sage: c2 = Cone([(-1,-2,1), (-1,2,1), (1,2,1), (1,-2,1)])
        sage: refined_cones = _refine_arrangement_to_fan([c1, c2])
        sage: for cone in refined_cones: print(cone.rays())
        N(-1,  1, 1),
        N(-1, -1, 1),
        N( 1, -1, 1),
        N( 1,  1, 1)
        in 3-d lattice N
        N(1, -1, 1),
        N(1,  1, 1),
        N(2, -1, 1),
        N(2,  1, 1)
        in 3-d lattice N
        N(-2,  1, 1),
        N(-1, -1, 1),
        N(-1,  1, 1),
        N(-2, -1, 1)
        in 3-d lattice N
        N(-1, 1, 1),
        N(-1, 2, 1),
        N( 1, 1, 1),
        N( 1, 2, 1)
        in 3-d lattice N
        N(-1, -1, 1),
        N(-1, -2, 1),
        N( 1, -2, 1),
        N( 1, -1, 1)
        in 3-d lattice N
    """
    dual_lattice = cones[0].dual_lattice()
    is_face_to_face = True
    for i in range(len(cones)):
        ci = cones[i]
        for j in range(i):
            cj = cones[j]
            c = ci.intersection(cj)
            if not (c.is_face_of(ci)) or not (c.is_face_of(cj)):
                is_face_to_face = False
                break
        if not is_face_to_face:
            break
    if is_face_to_face:
        return cones
    facet_normal_vectors = []
    for c in cones:
        for l in c.polyhedron().Hrepresentation():
            v = l[1::]
            is_new = True
            for fnv in facet_normal_vectors:
                if span([v, fnv]).dimension() < 2:
                    is_new = False
                    break
            if is_new:
                facet_normal_vectors.append(v)
    for v in facet_normal_vectors:
        halfspace1 = Cone([v], lattice=dual_lattice).dual()
        halfspace2 = Cone([-v], lattice=dual_lattice).dual()
        subcones = []
        for c in cones:
            subc1 = c.intersection(halfspace1)
            subc2 = c.intersection(halfspace2)
            for subc in [subc1, subc2]:
                if subc.dim() == c.dim():
                    is_new = True
                    for subcone in subcones:
                        if subc.dim() == subcone.dim() and subc.is_equivalent(subcone):
                            is_new = False
                            break
                    if is_new:
                        subcones.append(subc)
        cones = subcones
    return cones

In [6]:
from sage.misc.prandom import random, choice
import time

In [14]:
dim = 5
num = 5
rays_in_cone = 5
nrays = 50
ntests = 1000

def inner_product(x,y):
    return sum([x[_]*y[_] for _ in range(len(x))])

direction_ray = tuple(choice([-1,1])*randint(1,100) for i in range(dim))
generating_cones = []
i = 0
rays = []
while i<nrays:
    ray = tuple(choice([-1,1])*randint(1,100) for i in range(dim))
    if inner_product(direction_ray, ray) <= 0:
        continue
    else:
        rays.append(ray)
        i+=1

generating_cones = []
for i in range(num):
    c_nrays = choice([1 .. rays_in_cone])
    c_rays = []
    for j in [1 .. c_nrays]:
        c_rays.append(rays[choice([0..nrays-1])])
    c = Cone(rays = c_rays)
    generating_cones.append(c)
        
testing_cone = []
for i in range(ntests):
    c_nrays = choice([1 .. rays_in_cone])
    c_rays = []
    for j in [1 .. c_nrays]:
        c_rays.append(rays[choice([0..nrays-1])])
    c = Cone(rays = c_rays)
    testing_cone.append(c)

In [15]:
testing_points = []
for i in range(1000):
    ray = tuple(randint(-1000,1000) for i in range(dim))
    testing_points.append(ray) 

In [16]:
tik = time.time()
D = Fan(generating_cones,discard_faces=True,allow_arrangement=True)
tok = time.time()
print("time is {}".format(tok-tik))

tik = time.time()
C = Fan_rtree(generating_cones,discard_faces=True,allow_arrangement=True)
tok = time.time()
print("time is {}".format(tok-tik))

print(len(C.generating_cones()))

time is 102.33762097358704
Before rtree60.47275185585022s.
time is 113.38309407234192
2728


In [17]:
@timeit
def D_result(L):
    for c in testing_points:
        L.append(D.support_contains(c))
        
@timeit
def C_result(L):
    for c in testing_points:
        L.append(C.support_contains(c))

In [18]:
E = []
F = []
D_result(E)
C_result(F)

time is 140.72904682159424
time is 55.95819425582886


In [21]:
if all(E[i]==F[i] for i,x in enumerate(E)):
    print("good!")

good!


In [22]:
C._rtree

rtree.index.Index(bounds=[-67066469.0, 22895649.0, -44152650.0, 66957976.0, -67107742.0, 24429166.0, -33160376.0, 67059953.0, -26333891.0, 67082013.0], size=2728)